# breast_cancer resuelto SIN la librería sklearn
Normalizando los datos.

In [1]:
import numpy as np
import pandas as pd
from tabulate import tabulate

## Funciones para la normalización, entrenamiento y predicción

In [2]:
class StandardScalerManual:
    def __init__(self):
        self.mean_ = None
        self.scale_ = None
    
    def fit(self, X):
        """Calcula la media y desviación estándar de X."""
        self.mean_ = np.mean(X, axis=0)
        self.scale_ = np.std(X, axis=0)
        return self
    
    def transform(self, X):
        """Normaliza X usando la media y desviación estándar."""
        return (X - self.mean_) / self.scale_
    
    def fit_transform(self, X):
        """Ajusta el scaler a X y devuelve X normalizado."""
        return self.fit(X).transform(X)

class PerceptronManual:
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights = None
        self.bias = None
    
    def fit(self, X, y):
        """Entrena el perceptrón usando el conjunto de datos X e y."""
        n_samples, n_features = X.shape
        
        # Inicializar pesos y bias
        self.weights = np.zeros(n_features)
        self.bias = 0
        
        # Entrenamiento
        for _ in range(self.n_iterations):
            for idx, x_i in enumerate(X):
                # Predicción
                linear_output = np.dot(x_i, self.weights) + self.bias
                y_predicted = 1 if linear_output > 0 else 0
                
                # Actualización de pesos si hay error
                if y_predicted != y[idx]:
                    update = self.learning_rate * (y[idx] - y_predicted)
                    self.weights += update * x_i
                    self.bias += update
    
    def predict(self, X):
        """Realiza predicciones para el conjunto X."""
        linear_output = np.dot(X, self.weights) + self.bias
        return np.where(linear_output > 0, 1, 0)

def train_test_split_manual(X, y, test_size=0.2, random_state=None):
    """División manual de datos en conjuntos de entrenamiento y prueba."""
    if random_state is not None:
        np.random.seed(random_state)
    
    n_samples = len(X)
    n_test = int(n_samples * test_size)
    
    # Crear índices aleatorios
    indices = np.random.permutation(n_samples)
    test_indices = indices[:n_test]
    train_indices = indices[n_test:]
    
    # Dividir los datos
    X_train = X[train_indices]
    X_test = X[test_indices]
    y_train = y[train_indices]
    y_test = y[test_indices]
    
    return X_train, X_test, y_train, y_test

def accuracy_score_manual(y_true, y_pred):
    """Calcula la precisión del modelo."""
    return np.mean(y_true == y_pred)

def classification_report_manual(y_true, y_pred):
    """Genera un reporte de clasificación manual."""
    # Calcular verdaderos positivos, falsos positivos, etc.
    tp = np.sum((y_true == 1) & (y_pred == 1))
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    
    # Calcular métricas
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    
    return {
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'accuracy': accuracy
    }

## Cargar, preparar y normalizar datos

In [3]:
# Cargar y preparar los datos
file_path = "../data/raw/data.csv"
df = pd.read_csv(file_path, header=None)

# Crear nombres cortos para las características
feature_names = [f'feat{str(i+1).zfill(2)}' for i in range(30)]

# Eliminar columna ID y asignar nombres a las columnas
df = df.drop([0], axis=1)  # El ID no aporta información
df.columns = ['diagnosis'] + feature_names

# Mapear diagnóstico de M/B a 0/1
df['diagnosis'] = df['diagnosis'].map({'M': 0, 'B': 1})

# Mostrar los primeros registros en formato tabla
print("\nPrimeros registros del dataset:\n")
print(tabulate(df.head(21), headers='keys', tablefmt='rst', showindex=True, floatfmt='.4f'))


Primeros registros del dataset:

====  ===========  ========  ========  ========  =========  ========  ========  ========  ========  ========  ========  ========  ========  ========  ========  ========  ========  ========  ========  ========  ========  ========  ========  ========  =========  ========  ========  ========  ========  ========  ========
  ..    diagnosis    feat01    feat02    feat03     feat04    feat05    feat06    feat07    feat08    feat09    feat10    feat11    feat12    feat13    feat14    feat15    feat16    feat17    feat18    feat19    feat20    feat21    feat22    feat23     feat24    feat25    feat26    feat27    feat28    feat29    feat30
====  ===========  ========  ========  ========  =========  ========  ========  ========  ========  ========  ========  ========  ========  ========  ========  ========  ========  ========  ========  ========  ========  ========  ========  ========  =========  ========  ========  ========  ========  ========  ========
   0  

### Dividir en características (X) y target (y)

In [4]:
# X variable con 30 características independientes, y con una característica dependiente
X = df[feature_names].values  # Convertir a numpy array
y = df['diagnosis'].values    # Convertir a numpy array

# Verificar las dimensiones
print("\nVerificación de dimensiones:")
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")


Verificación de dimensiones:
X shape: (569, 30)
y shape: (569,)


In [5]:
# Normalizar los datos
scaler = StandardScalerManual()
X_scaled = scaler.fit_transform(X)

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split_manual(X_scaled, y, test_size=0.2, random_state=42)

# Crear y entrenar el perceptrón
perceptron = PerceptronManual(learning_rate=0.01, n_iterations=1000)
perceptron.fit(X_train, y_train)

# Realizar predicciones
y_pred = perceptron.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score_manual(y_test, y_pred)
report = classification_report_manual(y_test, y_pred)

print("\nResultados del modelo:")
print(f"Precisión del modelo: {accuracy:.2f}")
print("Reporte de clasificación:")
print(report)


Resultados del modelo:
Precisión del modelo: 0.93
Reporte de clasificación:
{'precision': np.float64(0.9846153846153847), 'recall': np.float64(0.9014084507042254), 'f1_score': np.float64(0.9411764705882353), 'accuracy': np.float64(0.9292035398230089)}
